In [2]:
import pandas as pd
import requests
import numpy as np


In [3]:
pd.set_option('display.max_rows', 500)


In [27]:
def get_data_by_variable(variable_id : int) -> pd.DataFrame:
    headers = {
        'format': 'application/json',
        'lang': 'pl',
        'X-ClientId': '54f60e5f-cb69-4b70-fc64-08d9e6308fac',
    }
    response = requests.get(f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}/?page-size=100&page=0&unit-level=5', headers=headers)
    number_of_records = response.json()['totalRecords']
    data = {}

    for i in range(number_of_records // 100 + 1):
        api_call = requests.get(f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}/?unit-level=5&page-size=100&page={i}', headers=headers)
        json = api_call.json()

        for result in json['results']:
            years = [row['year'] for row in result['values']]
            values = [row['val'] for row in result['values']]
            data[result['name']] = pd.Series(data=values, index=years)

    return pd.DataFrame.from_dict(data, orient='index')

# Zmienne
<hr/>

\<nazwa zmiennej\> [id zmiennej]



### Sport
- kluby sportowe na 10 tys. ludnosci [1609348]

### Zarobki
- przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100) [64429]

### Ludnosc
- gminy bez miast na prawach powiatu ogółem 30.06 [199202]
- gminy bez miast na prawach powiatu ogółem 31.12 [199206]
- miasta na prawach powiatu ogółem 30.06 [199203]
- miasta na prawach powiatu ogółem 31.12 [199207]
### Turystyka
- udzielone noclegi na 1000 ludnosci ogółem [60298]
- turysci korzystający z noclegow na 1000 ludnosci ogółem [60297]




# Zarobki

In [69]:
zarobki = get_data_by_variable(64429)
zarobki = zarobki.loc[:, '2015':]
zarobki.dropna(inplace=True)


In [91]:
zarobki.dropna(inplace=True)

In [97]:
np.mean(zarobki['2020'] - zarobki['2019'])

0.6862162162162164

In [102]:
(zarobki['2020'] - 100).sort_values()

Powiat kępiński                  -29.9
Powiat nowomiejski               -28.1
Powiat krotoszyński              -27.2
Powiat złotowski                 -26.8
Powiat krośnieński               -26.7
Powiat strzyżowski               -26.6
Powiat kłobucki                  -26.0
Powiat lipnowski                 -25.6
Powiat brzeziński                -25.5
Powiat przemyski                 -25.3
Powiat zamojski                  -25.1
Powiat aleksandrowski            -25.1
Powiat tarnowski                 -24.9
Powiat łukowski                  -24.9
Powiat działdowski               -24.9
Powiat gorlicki                  -24.7
Powiat szydłowiecki              -24.7
Powiat nowosądecki               -24.7
Powiat limanowski                -24.6
Powiat łaski                     -24.6
Powiat wieruszowski              -24.6
Powiat koniński                  -24.2
Powiat jarociński                -24.1
Powiat kaliski                   -24.0
Powiat niżański                  -24.0
Powiat suwalski          

# Sport

In [71]:
kluby_sportowe = get_data_by_variable(1609348)


In [88]:
kluby_sportowe = kluby_sportowe.loc[:, '2014':]

In [103]:
kluby_sportowe.agg(np.mean, axis=1)
# np.mean(kluby_sportowe['2020'] - kluby_sportowe['2016'])

Powiat bocheński                  4.800
Powiat krakowski                  4.050
Powiat miechowski                 5.425
Powiat myślenicki                 5.500
Powiat proszowicki                2.650
Powiat wielicki                   4.675
Powiat m.Kraków                   2.250
Powiat gorlicki                   6.225
Powiat limanowski                 5.775
Powiat nowosądecki                4.550
Powiat m.Nowy Sącz                5.075
Powiat chrzanowski                3.075
Powiat olkuski                    3.875
Powiat oświęcimski                4.325
Powiat wadowicki                  4.775
Powiat brzeski                    4.850
Powiat dąbrowski                  3.875
Powiat tarnowski                  4.950
Powiat m.Tarnów                   3.850
Powiat nowotarski                 5.325
Powiat suski                      3.375
Powiat tatrzański                 5.225
Powiat bielski                    2.650
Powiat cieszyński                 3.950
Powiat żywiecki                   5.200


# Turystyka

In [81]:
udzielone_noclegi = get_data_by_variable(60298)
turysci = get_data_by_variable(60297)

In [82]:
turysci = turysci.loc[:, '2015':]
turysci.dropna(inplace=True)

In [112]:
turysci_suma = turysci.agg(np.sum, axis=1)

# Ludność

Dane są aktualizowane co pół roku, użyjemy danych z czerwca. Ponadto dane są podzielone na powiaty i miasta na prawach powiatu.

In [61]:
ludnosc_gminy_czerwiec = get_data_by_variable(199202)
ludnosc_miasta_czerwiec = get_data_by_variable(199203)

miasta = ludnosc_miasta_czerwiec.index.to_series().filter(regex='m\\.')
nie_miasta = set(ludnosc_miasta_czerwiec.index).difference(miasta)

ludnosc_gminy_czerwiec.drop(index=miasta, inplace=True)
ludnosc_miasta_czerwiec.drop(index=nie_miasta, inplace=True)

ludnosc_czerwiec = pd.concat([ludnosc_miasta_czerwiec, ludnosc_gminy_czerwiec])
ludnosc_czerwiec = ludnosc_czerwiec.loc[:, '2015':]
ludnosc_czerwiec.dropna(inplace=True)


(372, 26)
(372, 26)


In [66]:
ludnosc_czerwiec

,2015,2016,2017,2018,2019,2020
Powiat m.Kraków,762508.0,762448.0,766739.0,769498.0,774839.0,780981.0
Powiat m.Nowy Sącz,83820.0,83829.0,83947.0,83958.0,83813.0,83762.0
Powiat m.Tarnów,110956.0,110381.0,109842.0,109358.0,108580.0,108177.0
Powiat m.Bielsko-Biała,172781.0,172407.0,171828.0,171277.0,170953.0,170303.0
Powiat m.Bytom,171515.0,170059.0,168968.0,167672.0,165975.0,164447.0
Powiat m.Piekary Śląskie,56588.0,56126.0,55820.0,55450.0,55088.0,54860.0
Powiat m.Częstochowa,229086.0,227270.0,225313.0,223322.0,221252.0,219278.0
Powiat m.Gliwice,183970.0,182969.0,181715.0,180708.0,179154.0,178186.0
Powiat m.Zabrze,176825.0,175882.0,175016.0,173784.0,172806.0,171691.0
Powiat m.Chorzów,110125.0,109541.0,109151.0,108668.0,107963.0,107443.0


In [65]:
ludnosc_czerwiec = pd.concat([ludnosc_miasta_czerwiec, ludnosc_gminy_czerwiec])
ludnosc_czerwiec = ludnosc_czerwiec.loc[:, '2015':]
ludnosc_czerwiec.dropna(inplace=True)
ludnosc_czerwiec.sort_values(by='2020')



,2015,2016,2017,2018,2019,2020
Powiat sejneński,20718.0,20531.0,20301.0,20171.0,20002.0,19807.0
Powiat bieszczadzki,22123.0,22032.0,21921.0,21866.0,21799.0,21576.0
Powiat węgorzewski,23531.0,23329.0,23193.0,22948.0,22796.0,22555.0
Powiat leski,26783.0,26704.0,26664.0,26577.0,26532.0,26413.0
Powiat gołdapski,27330.0,27191.0,27086.0,26987.0,26825.0,26625.0
Powiat łosicki,31895.0,31628.0,31455.0,31180.0,30895.0,30658.0
Powiat brzeziński,30922.0,30932.0,30886.0,30841.0,30818.0,30793.0
Powiat nidzicki,33764.0,33558.0,33352.0,33130.0,32940.0,32753.0
Powiat białobrzeski,33538.0,33553.0,33540.0,33534.0,33524.0,33332.0
Powiat kazimierski,34511.0,34322.0,34079.0,33921.0,33740.0,33489.0


In [110]:
ludnosc_czerwiec.agg(np.mean, axis=1)

Powiat m.Kraków                   7.695022e+05
Powiat m.Nowy Sącz                8.385483e+04
Powiat m.Tarnów                   1.095490e+05
Powiat m.Bielsko-Biała            1.715915e+05
Powiat m.Bytom                    1.681060e+05
Powiat m.Piekary Śląskie          5.565533e+04
Powiat m.Częstochowa              2.242535e+05
Powiat m.Gliwice                  1.811170e+05
Powiat m.Zabrze                   1.743340e+05
Powiat m.Chorzów                  1.088152e+05
Powiat m.Katowice                 2.963108e+05
Powiat m.Mysłowice                7.467067e+04
Powiat m.Ruda Śląska              1.385412e+05
Powiat m.Siemianowice Śląskie     6.750200e+04
Powiat m.Świętochłowice           5.031467e+04
Powiat m.Jastrzębie-Zdrój         8.948850e+04
Powiat m.Rybnik                   1.389170e+05
Powiat m.Żory                     6.224517e+04
Powiat m.Dąbrowa Górnicza         1.210395e+05
Powiat m.Jaworzno                 9.194817e+04
Powiat m.Sosnowiec                2.038343e+05
Powiat m.Tych

In [116]:
dane = {
    'ludnosc' : ludnosc_czerwiec.agg(np.mean, axis=1),
    'turystyka': turysci_suma,
    'kluby_sportowe': kluby_sportowe.agg(np.mean, axis=1),
    'zarobki': (zarobki['2020'] - 100)
}

dane_razem = pd.DataFrame(dane)


In [130]:
dane_razem

,ludnosc,turystyka,kluby_sportowe,zarobki,kluby_sportowe_lacznie,turysci_lacznie
Powiat aleksandrowski,5.529017e+04,15838.13,4.075,-25.1,22,875692
Powiat augustowski,5.860000e+04,6369.10,4.050,-9.3,23,373229
Powiat bartoszycki,5.834633e+04,1483.87,2.650,-16.9,15,86578
Powiat bełchatowski,1.129200e+05,2734.54,4.575,26.8,51,308784
Powiat bialski,1.117043e+05,4076.63,3.475,-22.9,38,455377
Powiat białobrzeski,3.350350e+04,1030.58,3.375,-20.1,11,34528
Powiat białogardzki,4.802350e+04,771.69,3.425,-15.4,16,37059
Powiat białostocki,1.474170e+05,2497.00,3.800,-19.3,56,368100
Powiat bielski,5.543917e+04,1503.15,2.650,0.4,14,83333
Powiat bieruńsko-lędziński,5.942450e+04,1229.77,3.400,-15.8,20,73078


In [129]:
dane_razem['kluby_sportowe_lacznie'] = dane_razem.kluby_sportowe * dane_razem.ludnosc / 10_000 // 1
dane_razem.kluby_sportowe_lacznie = dane_razem.kluby_sportowe_lacznie.astype(np.int64)

dane_razem['turysci_lacznie'] = dane_razem.turystyka * dane_razem.ludnosc / 1_000 // 1
dane_razem['turysci_lacznie'] = dane_razem['turysci_lacznie'].astype(np.int64)

In [135]:
zarobki_norm = (dane_razem.zarobki - dane_razem.zarobki.min()) / (dane_razem.zarobki.max() - dane_razem.zarobki.min())
zarobki_norm.sort_values()

dane_razem.zarobki.transform(lambda x: (x - x.min())/ (x.max() - x.min())).sort_values()

Powiat kępiński                   0.000000
Powiat nowomiejski                0.019694
Powiat krotoszyński               0.029540
Powiat złotowski                  0.033917
Powiat krośnieński                0.035011
Powiat strzyżowski                0.036105
Powiat kłobucki                   0.042670
Powiat lipnowski                  0.047046
Powiat brzeziński                 0.048140
Powiat przemyski                  0.050328
Powiat zamojski                   0.052516
Powiat aleksandrowski             0.052516
Powiat tarnowski                  0.054705
Powiat łukowski                   0.054705
Powiat działdowski                0.054705
Powiat gorlicki                   0.056893
Powiat nowosądecki                0.056893
Powiat szydłowiecki               0.056893
Powiat wieruszowski               0.057987
Powiat limanowski                 0.057987
Powiat łaski                      0.057987
Powiat koniński                   0.062363
Powiat jarociński                 0.063457
Powiat kali

In [131]:
dane_razem.zarobki.min()

-29.900000000000006